In [1]:
import numpy as np
from matplotlib import pyplot as plt


# width      = 1000 * 540 * 1.5 / 2
# spacesteps = 1000

# x = np.linspace(0, width, spacesteps)
# dx = width / spacesteps

In [2]:
from math import floor
from tqdm import tqdm

def trapezium(func, bounds, delta, display=False):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 2

    steps = (bounds[1] - bounds[0]) / delta

    if display:
        for i in tqdm(range(1, floor(steps))):
            result += delta * func(bounds[0] + (delta * i))

    else:
        for i in range(1, floor(steps)):
            result += delta * func(bounds[0] + (delta * i))

    return result
    
def simpson(func, bounds, delta):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 3

    steps = (bounds[1] - bounds[0]) / delta
    for i in range(1, floor(steps)):
        add = 2 * delta * func(bounds[0] + (delta * i))

        if i % 2 != 0:
            add *= 2

        result += add / 3

    return result

In [3]:
trap = trapezium(lambda x: x ** 2, (0, 10), 0.01)
simp = simpson  (lambda x: x ** 2, (0, 10), 0.01)
print(f'{trap} vs {simp}')

333.33349999999996 vs 333.33333333333337


In [4]:
from math import sqrt, pi, cos, sin
from cmath import exp

S = 3.6e-5
L = 10000
N = 0.01
rho_s = 1
D = 50000
D_t = 10000
h = 100000

mode = 1

# modal variables
A_j = sqrt(2 / (rho_s * (N ** 2) * D))

# wavespeed (squared)
c2 = ((N * D) ** 2) / (((mode * pi) ** 2) + ((D ** 2)/(4 * (h ** 2)))) # wavespeed

# calculating S_j = S0 * sigma
if ((mode * D_t / D) - 1) == 0:
    S_j = A_j * (rho_s / 2) * D_t
else:
    S_A = np.sin((np.pi) * ((mode * D_t / D) - 1)) / ((mode * D_t / D) - 1)
    S_B = np.sin((np.pi) * ((mode * D_t / D) + 1)) / ((mode * D_t / D) + 1)
    S_j = A_j * (rho_s / 2) * (D_t / np.pi) * (S_A - S_B)

S_j = S_j * S


def F(x):
    return 1 / np.cosh(x / L) ** 2

In [41]:
steps = 1000

def FourierTransform(func, k):
    xMax = 50 * L
    dx = xMax / steps
    # e^ix   = cos(x) + i*sin(x)
    # e^-ikx = e^i(-kx) = cos(-kx) + i*sin(-kx) = cos(kx) - i*sin(kx)

    function = lambda x: exp(k * x * -1j) * func(x) 

    return trapezium(function, (-xMax, xMax), dx)

def InverseTest(func, x):
    kMax = 30 / L
    dk = kMax / steps

    function = lambda k: exp(k * x * 1j) * func(k)

    return trapezium(function, (-kMax, kMax), dk, True) / (2 * pi) 


In [ ]:
for x in [t * 1000 for t in [300, 405]]:
    true = F(x)

    #fR = lambda k: FourierR(F, k)
    #fI = lambda k: FourierI(F, k)

    #fourier = InverseFourierTransform((fR, fI), x)

    f = lambda k: FourierTransform(F, k)
    fourier = InverseTest(f, x)

    print(f'x={x}: {true} | {fourier} | {str(np.round(100 * (true - fourier) / true, 2))}%')

In [ ]:
a = 5
example      = lambda x: exp(-a * abs(x))
exampleTilde = lambda k: (2 * a) / ((a ** 2) + k ** 2)

testTildeR   = lambda k: FourierR(example, k)
testTildeI   = lambda k: FourierI(example, k)


for k in [0, 1, 5, 10, 50, 100]:
    print(f'Fourier values: (k={k})')
    true    = exampleTilde(k)
    fourier = testTildeR(k) # we know im should be 0

    print(f'{true} | {fourier} | {str(round(100 * (true - fourier) / true, 2))}%')

    print(f'Inverted values: (x={k})')
    true     = example(k)
    inverted = InverseFourierTransform((testTildeR, testTildeI), k)

    print(f'{true} | {inverted} | {str(round(100 * (true - inverted[0]) / true, 2))}%')

    print(f'------------------------------')
